In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import py3Dmol

from proteome import protein
from proteome.models.design.proteinmpnn import config
from proteome.models.design.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

In [3]:
designer = ProteinMPNNForSequenceDesign("vanilla_model-20", random_seed=37)
folder = OmegaFoldForFolding()

In [4]:
with open("5L33.pdb", mode="r") as f:
    gt_pdb = f.read()

In [5]:
target_protein = protein.from_pdb_string(
    gt_pdb, ca_only=designer.ca_only, backbone_only=(not designer.ca_only)
)

In [6]:
sequence, score = designer.design_sequence(target_protein)
print(f"Sequence: {sequence} with global_score {score}")

Sequence: MVDEETKKALDFIEAMEKADPELMKKVITPDTEMNVNGKKFKGEEIVEMMKKHKEKGVKFELESYEKKGDKYVFKVKVSKKGKEKKATVEITVKDGKIEKIEIKEE with global_score 0.8684336543083191


In [7]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [8]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol